---
title: Project Pipeline
description: Project pipeline produces dataset for analysis for a specific purpose (project).
---

Generally, it includes the following steps:

- Combining data/events from different missions into a single dataset for analysis.

L1 level solar wind dataset

- Remove extreme values
- Select data without the influence of the planetary magnetosphere

In [ ]:
#| default_exp pipelines/project/pipeline
#| export
from datetime import timedelta
import polars as pl
import polars.selectors as cs
from beforerr.basics import pmap

from kedro.pipeline import Pipeline, node
from kedro.pipeline.modular_pipeline import pipeline

from loguru import logger

In [1]:
#| output: False
from ids_finder.utils.basic import load_catalog

catalog = load_catalog('../../')

[11/16/23 21:44:47] WARNING  /Users/zijin/miniforge3/envs/cool_planet/lib/python3.10/site-packages/ ]8;id=978598;file:///Users/zijin/miniforge3/envs/cool_planet/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=376018;file:///Users/zijin/miniforge3/envs/cool_planet/lib/python3.10/warnings.py#109\109]8;;\
                             kedro_datasets/polars/lazy_polars_dataset.py:14:                                      
                             KedroDeprecationWarning: 'AbstractVersionedDataSet' has been renamed                  
                             to 'AbstractVersionedDataset', and the alias will be removed in Kedro                 
                             0.19.0                                                                                
                               from kedro.io.core import (                                                         
                                                                                                                   

In [11]:
# | export
from ids_finder.utils.analysis import filter_before_jupiter


def process_events_l1(events: pl.LazyFrame):
    "clean data to remove extreme values"
    events = events.collect()

    df = (
        events.pipe(filter_before_jupiter)
        .filter(
            pl.col("d_star") < 100,  # exclude extreme values
            pl.col("v_mn") > 10,
            pl.col("duration") < timedelta(seconds=60),
        )
        .with_columns(
            cs.float().cast(pl.Float64),
            j0_norm_log=pl.col("j0_norm").log10(),
            L_mn_norm_log=pl.col("L_mn_norm").log10(),
            k_x=pl.col("normal_direction").list.get(0),
            k_y=pl.col("normal_direction").list.get(1),
            k_z=pl.col("normal_direction").list.get(2),
        )
        .fill_nan(None)
    )

    logger.info(
        f"candidates_l1: {len(df)}, with effective ratio: {len(df) / len(events):.2%}"
    )

    return df.lazy()

In [ ]:
#| export
def create_l1_node(sat="JNO", ts=1, tau=60):
    ts_str = f"ts_{ts}s"
    tau_str = f"tau_{tau}s"
    return node(
        process_events_l1,
        inputs=f"events.{sat}_{ts_str}_{tau_str}",
        outputs=f"events.l1.{sat}_{ts_str}_{tau_str}",
    )

In [ ]:
# | export
def combine_events(**datasets):
    datasets = [v.with_columns(sat=pl.lit(key)) for key, v in datasets.items()]
    combined_dataset = pl.concat(datasets, how="diagonal")
    return combined_dataset.with_columns(
        pl.col("radial_distance").fill_null(1),  # by default, fill with 1 AU
    ).with_columns(
        r_bin=pl.col("radial_distance").round(),
    )


def create_pipeline():
    combine_layer = "events.l1"
    node_combine_events = node(
        combine_events,
        inputs=dict(
            JNO=f"{combine_layer}.JNO_ts_1s_tau_60s",
            STA=f"{combine_layer}.STA_ts_1s_tau_60s",
            THB=f"{combine_layer}.THB_sw_ts_1s_tau_60s",
        ),
        outputs=f"{combine_layer}.ALL_sw_ts_1s_tau_60s",
        # namespace="events.l1",
    )

    nodes = [
        create_l1_node("JNO"),
        create_l1_node("STA"),
        create_l1_node("THB_sw"),
        node_combine_events,
    ]
    return pipeline(nodes)

## Level 2 datasets

Time averaged solar wind dataset

In [ ]:
#| export
from ids_finder.utils.analysis import link_coord2dim, n2_normalize

In [ ]:
#| export
NVARS = ['d_star', 'L_mn', 'L_mn_norm', 'j0', 'j0_norm', 'duration', 'v_mn']

In [ ]:
# | export
def time_average(raw_df: pl.DataFrame, avg_window=timedelta(days=30)):
    """Time average"""
    time_col = "time"

    return (
        raw_df.sort(time_col)
        .group_by_dynamic(time_col, every=avg_window, by="sat")
        .agg(cs.numeric().mean(), cs.duration().mean(), id_count=pl.count())
        .filter(
            pl.col("id_count") > 25
        )  # make sure each group has enough data, filter out JUNO extreme large thickness
        .sort(time_col)
        .upsample(time_col, every=avg_window, by="sat", maintain_order=True)
        .with_columns(pl.col("sat").forward_fill())
    )


def process_events_l2(raw_df: pl.DataFrame, avg_window=timedelta(days=30)):
    """L2 level datasets
    - Time average
    - Link time and radial distance
    """

    return (
        raw_df.pipe(time_average, avg_window=avg_window)
        .pipe(link_coord2dim)
        .pipe(n2_normalize, cols=NVARS)
    )

In [ ]:
def combine_candidates(dict):
    pass

# node_thm_extract_features = node(
#     extract_features,
#     inputs=["primary_thm_rtn_1s", "params:tau", "params:thm_1s_params"],
#     outputs="candidates_thm_rtn_1s",
#     name="extract_ARTEMIS_features",
# )

# node_combine_candidates = node(
#     combine_candidates,
#     inputs=dict(
#         sta_candidates="candidates_sta_rtn_1s",
#         jno_candidates="candidates_jno_ss_se_1s",
#         thm_candidates="candidates_thm_rtn_1s",
#     ),
#     outputs="candidates_all_1s",
#     name="combine_candidates",
# )